# Star Wars Knowledge Graph

Neo4j: [192.168.68.40](http://192.168.68.40:7474/)

In [2]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.19.0-py3-none-any.whl size=280741 sha256=2d62e81c69ccde366b8a907824bf2debc0ba3c475425fbc36940fa48ceef8c0e
  Stored in directory: /Users/david.lozzi/Library/Caches/pip/wheels/48/0d/71/a6d5ad9ecdd1cb0fae2aeea918a9bed3d8120ded12b3cfa8ed
Successfully built neo4j


## Schema

In [3]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
load_dotenv()
password = os.getenv("DB_PASSWORD")

uri = "bolt://192.168.68.40:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", password))



In [8]:

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]

def pretty_print_path(path_record):
    path = path_record['path']
    nodes = path.nodes
    relationships = path.relationships

    output = []
    for i, node in enumerate(nodes):
        output.append(f"Node {i+1}: {dict(node)}")
        if i < len(relationships):
            rel = relationships[i]
            output.append(f"  -> Relationship: {rel.type} ->")

    return "\n".join(output)


In [4]:
schema_queries = [
    "CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;",
    "CREATE CONSTRAINT ON (l:Location) ASSERT l.name IS UNIQUE;",
    "CREATE CONSTRAINT ON (v:Vehicle) ASSERT v.name IS UNIQUE;",
    "CREATE CONSTRAINT ON (m:Media) ASSERT m.title IS UNIQUE;"
]

for query in schema_queries:
    run_query(query)


In [5]:
# Test creating a node
test_query = "CREATE (c:Character {name: 'Test Character'}) RETURN c"
result = run_query(test_query)
print(result)

# View all constraints in the database
constraints_query = "CALL db.constraints()"
constraints_result = run_query(constraints_query)
print(constraints_result)


[<Record c=<Node element_id='0' labels=frozenset({'Character'}) properties={'name': 'Test Character'}>>]
[<Record name='constraint_1589324c' description='CONSTRAINT ON ( character:Character ) ASSERT (character.name) IS UNIQUE' details="Constraint( id=4, name='constraint_1589324c', type='UNIQUENESS', schema=(:Character {name}), ownedIndex=3 )">, <Record name='constraint_2973dcd5' description='CONSTRAINT ON ( location:Location ) ASSERT (location.name) IS UNIQUE' details="Constraint( id=6, name='constraint_2973dcd5', type='UNIQUENESS', schema=(:Location {name}), ownedIndex=5 )">, <Record name='constraint_96f270e7' description='CONSTRAINT ON ( vehicle:Vehicle ) ASSERT (vehicle.name) IS UNIQUE' details="Constraint( id=8, name='constraint_96f270e7', type='UNIQUENESS', schema=(:Vehicle {name}), ownedIndex=7 )">, <Record name='constraint_c352c9b4' description='CONSTRAINT ON ( media:Media ) ASSERT (media.title) IS UNIQUE' details="Constraint( id=10, name='constraint_c352c9b4', type='UNIQUENESS'

In [20]:
# Create Obi-Wan Kenobi and relate him to a movie
create_obiwan_and_relationship = """
CREATE (lea:Character {name: "Princess Leia Organa", species: "Human"})
MERGE (newHope:Movie {title: "Star Wars: A New Hope"})
CREATE (lea)-[:APPEARED_IN]->(newHope)
"""
run_query(create_obiwan_and_relationship)


[]

: 

In [14]:
# Query to find all paths between Luke Skywalker and Anakin Skywalker
find_paths_query = """
MATCH path = (obiwan:Character {name: "Obi-wan Kenobi"})-[:APPEARED_IN*]-(movie:Movie)-[:APPEARED_IN*]-(anakin:Character {name: "Anakin Skywalker"})
RETURN path
"""

# Execute the query to find paths
paths = run_query(find_paths_query)

# Pretty print each path found
for path in paths:
    print(pretty_print_path(path))
    print("\n")

In [19]:
# Check if Obi-Wan Kenobi node exists
exists_query = "MATCH (o:Character {name: 'Obi-Wan Kenobi'}) RETURN o"
obiwan_exists = run_query(exists_query)
print(obiwan_exists)

# Check if Obi-Wan Kenobi has APPEARED_IN relationships
relationships_query = "MATCH (o:Character {name: 'Obi-Wan Kenobi'})-[:APPEARED_IN]->(m:Movie) RETURN o.name, collect(m.title)"
obiwan_relationships = run_query(relationships_query)
print(obiwan_relationships)

[<Record o=<Node element_id='9' labels=frozenset({'Character'}) properties={'species': 'Human', 'name': 'Obi-Wan Kenobi'}>>]
[<Record o.name='Obi-Wan Kenobi' collect(m.title)=['Star Wars: A New Hope', 'Star Wars: The Phantom Menace']>]
